In [2]:
import pandas as pd
import ast #To read strings back into lists/dicts
import  copy

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_language        3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null object
runtime                  2998 non-null float64
spoken_languages         2980 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null object
Keywords             

In [3]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [49]:
#This column only has 1 relevant key but several relevant values attached to that key.
train['genres'][1]

"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]"

In [7]:
#Some of these columns have relevant info in more than one key.
#i.e. 'cast' has relevant info in the 'character' and 'name' keys.
train['cast'][0][:400]

"[{'cast_id': 4, 'character': 'Lou', 'credit_id': '52fe4ee7c3a36847f82afae7', 'gender': 2, 'id': 52997, 'name': 'Rob Corddry', 'order': 0, 'profile_path': '/k2zJL0V1nEZuFT08xUdOd3ucfXz.jpg'}, {'cast_id': 5, 'character': 'Nick', 'credit_id': '52fe4ee7c3a36847f82afaeb', 'gender': 2, 'id': 64342, 'name': 'Craig Robinson', 'order': 1, 'profile_path': '/tVaRMkJXOEVhYxtnnFuhqW0Rjzz.jpg'}, {'cast_id': 6, "

In [16]:
def get_column_info(series, keys):
    '''
    Allows returned column to replacd the relevant df column in place
    or as a new Series. Allows for further counting with the gather function.
    
    Parameters:
    series (pd.Series) The relevant column from the DataFrame.
    keys (list) Can be a list of just 1 or several keys to search for.
    
    Returns:
    column (pd.Series) Each row has a new dict with lists of values associated 
    with each given key.
    '''
    column = copy.deepcopy(series)
    for index, data in column.iteritems():
        #Populate this row of the new df with the new dict
        #Calls the get_info method
        column.at[index] = get_info(data, keys)
    return column

def get_info(row, keys):
    '''
    Pulls the values from each key into a new list for each key.
    
    Parameters:
    row (str) String to be evaluated as list of dicts.
    keys (list) Can be a list of just 1 or several keys to search for.
    
    Returns:
    info (dict) The values of the relevant keys as new lists.
    '''
    #if the data can be turned into a list or dict
    if row != '':
        #New empty dict of lists
        info = {}
        #For each of the provided keys 
        for key in keys:
            #
            info[key] = []
            #Turn data into list of dicts
            list_of_dicts = ast.literal_eval(row)
            #Read each dict
            for dic in list_of_dicts:
                #Add the info from the provided key to the list of info
                info[key].append(dic[key])
        #Return the list of info matching the key from every row
        return info

def gather(column, keys):
    '''
    Gathers up all values sassociated with each key across all rows
    in the column into a single list. Each list is added to a new dict.
    
    Parameters:
    column (pd.Series) Column of rows to iterate over.
    
    keys (list) Can be a list of just 1 or several keys to search for.
    
    Returns:
    info_lists (dict) Dictionaries of lists of values associated with each key.
    '''
    #New dict of lists
    info_lists = {}
    
    #New empty list for each key in keys
    for key in keys:
        info_lists[key] = [] 
    
    #Iterate over each key in the list. Allows for one or several keys.
    for index, data in column.iteritems():
        for key in keys:
            #If row isn't blank
            if data:
                #Add the values to the list for that key
                info_lists[key].extend(data[key])
    
    return info_lists

In [19]:
#Get the value counts for 'name' key in each column.
#Also get the separate 'character' and 'actor' counts 
#from the 'cast' column.
%%time
belongs_to_collection = get_column_info(train['belongs_to_collection'].fillna(''), ['name'])
collection_counts = pd.Series(gather(belongs_to_collection, ['name'])['name']).value_counts()

genres = get_column_info(train['genres'].fillna(''), ['name'])
genre_counts = pd.Series(gather(genres, ['name'])['name']).value_counts()

production_companies = get_column_info(train['production_companies'].fillna(''), ['name'])
company_counts = pd.Series(gather(production_companies, ['name'])['name']).value_counts()

production_countries = get_column_info(train['production_countries'].fillna(''), ['name'])
country_counts = pd.Series(gather(production_countries, ['name'])['name']).value_counts()

spoken_languages = get_column_info(train['spoken_languages'].fillna(''), ['name'])
language_counts = pd.Series(gather(spoken_languages, ['name'])['name']).value_counts()

keywords = get_column_info(train['Keywords'].fillna(''), ['name'])
keyword_counts = pd.Series(gather(keywords, ['name'])['name']).value_counts()

cast = get_column_info(train['cast'].fillna(''), ['character', 'name'])
actor_counts = pd.Series(gather(cast, ['name'])['name']).value_counts()
character_counts = pd.Series(gather(cast, ['character'])['character']).value_counts()

CPU times: user 4.68 s, sys: 39.3 ms, total: 4.72 s
Wall time: 4.75 s


In [59]:
genre_counts

Drama              1531
Comedy             1028
Thriller            789
Action              741
Romance             571
Crime               469
Adventure           439
Horror              301
Science Fiction     290
Family              260
Fantasy             232
Mystery             225
Animation           141
History             132
Music               100
War                 100
Documentary          87
Western              43
Foreign              31
TV Movie              1
dtype: int64

In [23]:
print(collection_counts.shape, 'different collections')
collection_counts[:20]

(422,) different collections


James Bond Collection                        16
Friday the 13th Collection                    7
The Pink Panther (Original) Collection        6
Police Academy Collection                     5
Pokémon Collection                            5
Resident Evil Collection                      4
Paranormal Activity Collection                4
Alien Collection                              4
Transformers Collection                       4
Ice Age Collection                            4
Rocky Collection                              4
The Fast and the Furious Collection           4
Child's Play Collection                       4
Rambo Collection                              4
The Vengeance Collection                      3
The Dark Knight Collection                    3
Three Heroes Collection                       3
Alex Cross Collection                         3
[REC] Collection                              3
Star Trek: The Original Series Collection     3
dtype: int64

In [43]:
#How clean are these collection names? Is there any redundancy to clean up?
print(collection_counts[collection_counts.index.str.contains('Star Trek')])
print(collection_counts[collection_counts.index.str.contains('Star Wars')])

Star Trek: The Original Series Collection    3
Star Trek: The Next Generation Collection    1
dtype: int64
Star Wars Collection    2
dtype: int64


In [25]:
print(actor_counts.shape, 'different actors')
actor_counts[:20]

(38588,) different actors


Samuel L. Jackson     30
Robert De Niro        30
Morgan Freeman        27
J.K. Simmons          25
Liam Neeson           25
Bruce Willis          25
Susan Sarandon        25
John Turturro         24
Bruce McGill          24
Willem Dafoe          23
Forest Whitaker       23
Nicolas Cage          22
Bill Murray           22
Owen Wilson           22
Jason Statham         21
Keith David           21
Mel Gibson            21
Sylvester Stallone    21
Sigourney Weaver      21
John Goodman          21
dtype: int64

In [46]:
#Are there any typos e.g. 'William Dafoe'
actor_counts[actor_counts.index.str.contains('Dafoe')]

Willem Dafoe    23
dtype: int64

In [61]:
character_counts.shape

(44422,)